In [1]:
from eda_import import *

In [4]:
%%time
with gzip.open('ls.genomecov.txt.gz', 'rt') as inf:
    inputs = [
        os.path.join(os.path.dirname(_), 'genomecov-by-gene.csv')
        for _ in inf.readlines()
    ]


dfs = []
for k, infile in enumerate(inputs):
    if (k + 1) % 500 == 0:
        print(k + 1, end=',')
    df = pd.read_csv(infile)
    df['source_file'] = os.path.abspath(infile)
    dfs.append(df)

500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10000,10500,CPU times: user 18.8 s, sys: 1.95 s, total: 20.7 s
Wall time: 1min 2s


In [5]:
%time combined = pd.concat(dfs)

CPU times: user 6.51 s, sys: 184 ms, total: 6.69 s
Wall time: 6.64 s


In [6]:
combined.head(1)

,gene_name,cov_average_num_bases,source_file
0,ABL1,83.905369,/projects/btl/zxue/tasrkleat-TCGA-results/tasrkleat-TCGA-analysis-scripts/RPKMS-calculation/tcga/KICH/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_2186137.d42c9589-3960-4964-be28-83d275bdc672.121106_UNC11-SN627_0258_AC11WRACXX_6_TGACCA.tar.gz/tasrkleat-results/align_reads2genome/genomecov-by-gene.csv


Load metadata, to obtain analysis_id

In [7]:
df_gcs_listing = pd.read_csv('./metadata/GCS_listing_27apr2016.csv.gz', compression='gzip')

In [8]:
df_gcs_listing.rename(columns={'CGHubAnalysisID':'analysis_id'}, inplace=True)

In [9]:
df_gcs_listing.head(1)

,analysis_id,AliquotBarcode,AliquotUUID,GCSobject,objectSizeBytes
0,00007994-abeb-4b16-a6ad-7230300a29e9,TCGA-AA-3495-01A-01R-1410-07,c18465ae-447d-46c8-8b54-0156ab502265,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga/COAD/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_1620885.c18465ae-447d-46c8-8b54-0156ab502265.sorted_genome_alignments.bam,1972948726


In [29]:
df_gcs_listing['source_file'] = df_gcs_listing.GCSobject.apply(
    lambda x: x.replace('gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/', 
                        '/projects/btl/zxue/tasrkleat-TCGA-results/tasrkleat-TCGA-analysis-scripts/RPKMS-calculation/') + '/tasrkleat-results/align_reads2genome/genomecov-by-gene.csv')

In [30]:
df_gcs_listing[df_gcs_listing.GCSobject.str.contains('SN627_0258_AC11WRACXX_6_TGACCA')]

,analysis_id,AliquotBarcode,AliquotUUID,GCSobject,objectSizeBytes,fname,source_file
9765,1d811d54-caa1-41f3-bfc2-400eebdd97ef,TCGA-KN-8425-01A-11R-2315-07,d42c9589-3960-4964-be28-83d275bdc672,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga/KICH/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_2186137.d42c9589-3960-4964-be28-83d275bdc672.121106_UNC11-SN627_0258_AC11WRACXX_6_TGACCA.tar.gz,8282670365,/projects/btl/zxue/tasrkleat-TCGA-results/tcga/KICH/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_2186137.d42c9589-3960-4964-be28-83d275bdc672.121106_UNC11-SN627_0258_AC11WRACXX_6_TGACCA.tar.gz/tasrkleat-results/align_reads2genome/genomecov-by-gene.csv,/projects/btl/zxue/tasrkleat-TCGA-results/tasrkleat-TCGA-analysis-scripts/RPKMS-calculation/tcga/KICH/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_2186137.d42c9589-3960-4964-be28-83d275bdc672.121106_UNC11-SN627_0258_AC11WRACXX_6_TGACCA.tar.gz/tasrkleat-results/align_reads2genome/genomecov-by-gene.csv


In [31]:
%%time
merged = combined.merge(
    df_gcs_listing[['source_file', 'analysis_id']].drop_duplicates(), how='left', on='source_file')

CPU times: user 502 ms, sys: 71.2 ms, total: 573 ms
Wall time: 571 ms


In [32]:
merged.shape

(1216152, 4)

In [34]:
merged.head(2)

,gene_name,cov_average_num_bases,source_file,analysis_id
0,ABL1,83.905369,/projects/btl/zxue/tasrkleat-TCGA-results/tasrkleat-TCGA-analysis-scripts/RPKMS-calculation/tcga/KICH/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_2186137.d42c9589-3960-4964-be28-83d275bdc672.121106_UNC11-SN627_0258_AC11WRACXX_6_TGACCA.tar.gz/tasrkleat-results/align_reads2genome/genomecov-by-gene.csv,1d811d54-caa1-41f3-bfc2-400eebdd97ef
1,AKT1,380.787244,/projects/btl/zxue/tasrkleat-TCGA-results/tasrkleat-TCGA-analysis-scripts/RPKMS-calculation/tcga/KICH/RNA/RNA-Seq/UNC-LCCC/ILLUMINA/UNCID_2186137.d42c9589-3960-4964-be28-83d275bdc672.121106_UNC11-SN627_0258_AC11WRACXX_6_TGACCA.tar.gz/tasrkleat-results/align_reads2genome/genomecov-by-gene.csv,1d811d54-caa1-41f3-bfc2-400eebdd97ef


In [36]:
%time merged.drop('source_file', axis=1).to_csv('./results/genomecov-by-gene-all-samples.csv', index=False)

CPU times: user 4.2 s, sys: 173 ms, total: 4.37 s
Wall time: 4.57 s
